<a href="https://colab.research.google.com/github/PreethiMachineMindsAcademy/Air-BNB-PricePrediction-Regression/blob/main/Welcome_To_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Welcome to Colab!

In [1]:
!pip install requests gradio

In [7]:
!pip install transformers torch accelerate gradio

In [5]:
!pip install gradio==3.50.2 requests --upgrade --force-reinstall

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.8/52.8 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.6/90.6 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.3/20.3 MB 65.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.2/299.2 kB 26.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.2/731.2 kB 50.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.7/153.7 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [21]:
def generate_pyspark_code(source_table):
    return f"""
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, month, sum

spark = SparkSession.builder \
    .appName("DealerMonthlySales") \
    .getOrCreate()

# Read Iceberg table
df = spark.read.format("iceberg").load("{source_table}")

# Handle null values
df_clean = df.filter(col("price").isNotNull())

# Extract month from sale_date
df_transformed = df_clean.withColumn("month", month(col("sale_date")))

# Aggregate total sales per dealer per month
df_agg = df_transformed.groupBy("dealer_id", "month") \
    .agg(sum("price").alias("total_sales"))

# Write output in Iceberg format partitioned by month
df_agg.write.format("iceberg") \
    .mode("overwrite") \
    .partitionBy("month") \
    .save("dealer_monthly_sales")

spark.stop()
"""


def run_agent(source, target):

    # ----- PLANNER (LLM Reasoning) -----
    plan_prompt = f"""
You are an expert Data Architect.

Task:
Design an ETL pipeline.

Source Table:
{source}

Target Requirement:
{target}

Provide:
1. Extraction step
2. Transformation steps
3. Aggregation logic
4. Storage format
5. Partition strategy

Answer in clear numbered steps.
"""
    plan = generate_response(plan_prompt)

    # ----- GENERATOR (Deterministic Template) -----
    source_table_name = "nsc_sales"
    code = generate_pyspark_code(source_table_name)

    # ----- REVIEWER (Optional LLM Refinement) -----
    review_prompt = f"""
Review this PySpark code and suggest improvements for:
- Performance
- Scalability
- Iceberg best practices

Code:
{code}

Return only suggestions (not rewritten code).
"""
    review_notes = generate_response(review_prompt)

    print("==== ETL PLAN (LLM) ====\n")
    print(plan)

    print("\n==== GENERATED PYSPARK CODE (Deterministic) ====\n")
    print(code)

    print("\n==== REVIEW SUGGESTIONS (LLM) ====\n")
    print(review_notes)

In [23]:
source = """
nsc_sales
dealer_id (int)
model_id (int)
price (double)
sale_date (date)
"""

target = """
Create monthly dealer sales summary.
Aggregate total sales per dealer per month.
Store result in Iceberg format.
"""

run_agent(source, target)

==== ETL PLAN (LLM) ====

1. Extraction 2. Transformation 3. Aggregation logic 4. Iceberg 5. Partition strategy

==== GENERATED PYSPARK CODE (Deterministic) ====


from pyspark.sql import SparkSession
from pyspark.sql.functions import col, month, sum

spark = SparkSession.builder     .appName("DealerMonthlySales")     .getOrCreate()

# Read Iceberg table
df = spark.read.format("iceberg").load("nsc_sales")

# Handle null values
df_clean = df.filter(col("price").isNotNull())

# Extract month from sale_date
df_transformed = df_clean.withColumn("month", month(col("sale_date")))

# Aggregate total sales per dealer per month
df_agg = df_transformed.groupBy("dealer_id", "month")     .agg(sum("price").alias("total_sales"))

# Write output in Iceberg format partitioned by month
df_agg.write.format("iceberg")     .mode("overwrite")     .partitionBy("month")     .save("dealer_monthly_sales")

spark.stop()


==== REVIEW SUGGESTIONS (LLM) ====

- Scalability and Iceberg best practices
